In [10]:
import numpy as np
import pandas as pd
 
data = pd.read_csv('stories.csv', sep = ';')
df = pd.DataFrame(data)
df.head(10)

,story_id,story_json
0,127,"['text"":""Как изменить ПИН-код', 'text"":""Зайдит..."
1,254,"['text"":""Как\xa0пополнить карту «Стрелка»', 't..."
2,865,"['text"":""Тинькофф Инвестиции\xa02.0', 'text"":""..."
3,1491,"['text"":""Выставка уличной культуры', 'text"":""1..."
4,598,"['text"":""Необычные виды спорта', 'text"":""Если ..."
5,202,"['text"":""Скрытые счета и\xa0карты', 'text"":""Мо..."
6,1405,"['text"":""Тинькофф-журнал отвечает на\xa0вопрос..."
7,195,"['text"":""РАЗВЛЕЧЕНИЯ', 'text"":""Лучшие места дл..."
8,1524,"['text"":""Рюкзаки для школы', 'text"":""Лучшие рю..."
9,205,"['text"":""Что такое овердрафт и\xa0зачем он\xa0..."


In [11]:
df.story_json = df.story_json.str.lower()
df.story_json = df.story_json.str.replace('[^a-zа-я0-9- ]', ' ')
df.story_json = df.story_json.str.replace('xa0', ' ')
df.story_json = df.story_json.str.replace('text', ' ')
df

,story_id,story_json
0,127,как изменить пин-код зайдите с гл...
1,254,как пополнить карту стрелка в ...
2,865,тинькофф инвестиции 2 0 у нас хо...
3,1491,выставка уличной культуры 18 и 19...
4,598,необычные виды спорта если бег б...
...,...,...
954,882,как компенсировать авиабилет милями ...
955,1473,как компенсировать покупку баллами брав...
956,266,как оплатить счет за коммуналку ...
957,764,плачу картой терминал просит подпись а...


In [12]:
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

In [13]:
df.story_json = df.story_json.apply(lemmatize)

In [14]:
df

,story_id,story_json
0,127,"[изменить, пин, код, зайти, главный, меню, выб..."
1,254,"[пополнить, карта, стрелка, приложение, тинько..."
2,865,"[тинькофф, инвестиция, хороший, новость, тот, ..."
3,1491,"[выставка, уличный, культура, август, пройти, ..."
4,598,"[необычный, вид, спорт, бег, бассейн, тренажёр..."
...,...,...
954,882,"[компенсировать, авиабилет, миля, зайти, вклад..."
955,1473,"[компенсировать, покупка, балл, браво, зайти, ..."
956,266,"[оплатить, счёт, коммуналка, ошибиться, реквиз..."
957,764,"[плач, карта, терминал, просить, подпись, пин,..."


In [9]:
df.to_csv('df_story_lem.csv', index=False)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
vec_text = CountVectorizer()
t = vec_text.fit_transform(df.story_json.astype(str))
df_text2 = pd.DataFrame({"Word": np.array(vec_text.get_feature_names()), 
                         'Kol':np.asarray(t.sum(axis=0)).ravel()})
df_text2 = (df_text2.sort_values(by=['Kol'], ascending=False))
df_text2 = df_text2.reset_index(drop=True)
df_text2

,Word,Kol
0,карта,846
1,это,672
2,который,648
3,год,581
4,купить,490
...,...,...
13910,проводы,1
13911,квадрокоптер,1
13912,кваркуш,1
13913,квартиросъёмщик,1


In [18]:
df_text2.to_csv('word.csv', index=False)
writer = pd.ExcelWriter('word.xlsx') 
df_text2.to_excel(writer) 
writer.save()
